In [2]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

df = pd.read_csv(r"C:\Users\Asus\Downloads\ML DATSETS\Uber\uber.csv")

In [11]:
# --- 1) Pre-process ---
df = df.dropna(subset=["fare_amount","pickup_datetime","pickup_longitude","pickup_latitude",
                       "dropoff_longitude","dropoff_latitude","passenger_count"])
print(f"✅ After removing NaN: {len(df):,}")

df = df[(df.fare_amount > 0) & (df.fare_amount < 500)]
df = df[(df.passenger_count >= 1) & (df.passenger_count <= 6)]
print(f"✅ After fixing fare & passenger count: {len(df):,}")

df = df[(df.pickup_latitude.between(40, 42)) &
        (df.dropoff_latitude.between(40, 42)) &
        (df.pickup_longitude.between(-75, -72)) &
        (df.dropoff_longitude.between(-75, -72))]
print(f"✅ After NYC geo-filter: {len(df):,}")

# Time features
dt = pd.to_datetime(df["pickup_datetime"], errors="coerce", utc=True)
df = df[dt.notna()].copy()
df["hour"] = dt.dt.hour
df["dow"]  = dt.dt.dayofweek

print("✅ Added time features (hour, day of week)")

# Distance calc (Haversine)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.009
    p1, p2 = np.radians(lat1), np.radians(lat2)
    dphi  = np.radians(lat2-lat1)
    dlmb  = np.radians(lon2-lon1)
    a = np.sin(dphi/2)**2 + np.cos(p1)*np.cos(p2)*np.sin(dlmb/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

df["dist_km"] = haversine(df.pickup_latitude, df.pickup_longitude,
                          df.dropoff_latitude, df.dropoff_longitude)

df = df[(df.dist_km > 0) & (df.dist_km < 200)]
print(f"✅ After distance cleanup: {len(df):,}")

✅ After removing NaN: 160,896
✅ After fixing fare & passenger count: 160,896
✅ After NYC geo-filter: 160,896
✅ Added time features (hour, day of week)
✅ After distance cleanup: 160,896


In [12]:
# --- 2) Outlier removal (IQR) ---
for col in ["fare_amount","dist_km"]:
    q1, q3 = df[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
    before = len(df)
    df = df[(df[col] >= low) & (df[col] <= high)]
    print(f"✅ Removed outliers in {col}: {before - len(df):,}")

print(f"📌 Final rows after cleaning: {len(df):,}\n")

✅ Removed outliers in fare_amount: 1,642
✅ Removed outliers in dist_km: 1,775
📌 Final rows after cleaning: 157,479



In [13]:
# --- 3) Correlation ---
feat = ["dist_km","hour","dow","passenger_count",
        "pickup_latitude","pickup_longitude","dropoff_latitude","dropoff_longitude"]

corr = df[feat + ["fare_amount"]].corr()["fare_amount"].sort_values(ascending=False)

print("📊 CORRELATION WITH FARE:\n", corr, "\n")

📊 CORRELATION WITH FARE:
 fare_amount          1.000000
dist_km              0.773065
hour                 0.019371
passenger_count      0.018642
dow                  0.006839
dropoff_longitude    0.003664
pickup_longitude    -0.023815
pickup_latitude     -0.062108
dropoff_latitude    -0.080450
Name: fare_amount, dtype: float64 



In [14]:
# --- 4) Split data ---
X, y = df[feat], df["fare_amount"]
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LinearRegression().fit(X_tr, y_tr)
rf = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42).fit(X_tr, y_tr)

In [15]:
# --- 5) Evaluation ---
def eval_model(name, mdl):
    p = mdl.predict(X_te)
    r2 = r2_score(y_te, p)
    rmse = np.sqrt(mean_squared_error(y_te, p))  # ✅ works in all sklearn versions

    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print(f"📌 Model: {name}")
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print(f"R² Score : {r2:.4f}")
    print(f"RMSE     : {rmse:.3f}")

eval_model("Linear Regression", lr)
eval_model("Random Forest", rf)

━━━━━━━━━━━━━━━━━━━━━━━━━━━
📌 Model: Linear Regression
━━━━━━━━━━━━━━━━━━━━━━━━━━━
R² Score : 0.6026
RMSE     : 1.924
━━━━━━━━━━━━━━━━━━━━━━━━━━━
📌 Model: Random Forest
━━━━━━━━━━━━━━━━━━━━━━━━━━━
R² Score : 0.6785
RMSE     : 1.730
